<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week07/Week7_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [2]:
# importing required libraries
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
import pyspark.sql.types as tp
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row, Column
import sys
from pyspark import SparkFiles

## Get prediction

In [3]:
# define the function to get the predicted sentiment on the data received
def get_prediction(tweet_text):
  try:
    
    # remove the blank tweets
    #print("remvoe the blank tweets")
    tweet_text = tweet_text.filter(lambda x: len(x) > 0)
    print("tweet_text ", tweet_text)
    
    # create the dataframe with each row contains a tweet text
    #print("create the dataframe")
    rowRdd = tweet_text.map(lambda w: Row(tweet=w))
    print("rowRdd ",rowRdd)

    wordsDataFrame = spark.createDataFrame(rowRdd)
    print("wordsDataFrame ",wordsDataFrame)

		# get the sentiments for each row
    pipelineFit.transform(wordsDataFrame).select('tweet','prediction').show()
  
  except : 
    print('No data')



In [4]:
sc = SparkContext(appName="PySparkShell")
spark = SparkSession(sc)

## Train model

In [5]:
#1 not good
#0 okay
url = "https://raw.githubusercontent.com/lakshay-arora/PySpark/master/spark_streaming/datasets/twitter_sentiments.csv"
spark.sparkContext.addFile(url)

# define the schema
my_schema = tp.StructType([
        tp.StructField(name= 'id',          dataType= tp.IntegerType(),  nullable= True),
        tp.StructField(name= 'label',       dataType= tp.IntegerType(),  nullable= True),
        tp.StructField(name= 'tweet',       dataType= tp.StringType(),   nullable= True)    
            ])
# reading the data set
print('\n\nReading the dataset...........................\n')
my_data = spark.read.csv("file://"+SparkFiles.get("twitter_sentiments.csv"), schema=my_schema, header=True)
my_data.show(5) # like head() in pandas

my_data.printSchema()
print('\n\nDefining the pipeline stages.................\n')
stage_1 = RegexTokenizer(inputCol= 'tweet' , outputCol= 'tokens', pattern= '\\W')
stage_2 = StopWordsRemover(inputCol= 'tokens', outputCol= 'filtered_words')
stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector', vectorSize= 100)

model = LogisticRegression(featuresCol= 'vector', labelCol= 'label') 

print('\n\nStages Defined................................\n')
pipeline = Pipeline(stages= [stage_1, stage_2, stage_3, model])

print('\n\nFit the pipeline with the training data.......\n')
pipelineFit = pipeline.fit(my_data)

print('\n\nModel Trained....Waiting for the Data!!!!!!!!\n')




Reading the dataset...........................

+---+-----+--------------------+
| id|label|               tweet|
+---+-----+--------------------+
|  1|    0| @user when a fat...|
|  2|    0|@user @user thank...|
|  3|    0|  bihday your maj...|
|  4|    0|#model   i love u...|
|  5|    0| factsguide: soci...|
+---+-----+--------------------+
only showing top 5 rows

root
 |-- id: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- tweet: string (nullable = true)



Defining the pipeline stages.................



Stages Defined................................



Fit the pipeline with the training data.......



Model Trained....Waiting for the Data!!!!!!!!



In [6]:
from pyspark.streaming.kafka import KafkaUtils
kafka_topic_name = "my-first-topic"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [7]:
ssc = StreamingContext(sc, 5)
kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

lines = kvs.map(lambda x: x[1])

words = lines.flatMap(lambda line : line.split(' '))
#words.pprint()

words.foreachRDD(get_prediction)

ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

tweet_text  PythonRDD[160] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[161] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[167] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[168] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[174] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[175] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[182] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[183] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[189] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[190] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[196] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[197] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[204] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[205] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[211] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[212] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[218] at RDD at PythonRDD.scala:53
rowRdd  PythonRD

KeyboardInterrupt: ignored

In [ ]:
ssc.stop()
sc.stop()

tweet_text  PythonRDD[226] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[227] at RDD at PythonRDD.scala:53
No data
tweet_text  PythonRDD[233] at RDD at PythonRDD.scala:53
rowRdd  PythonRDD[234] at RDD at PythonRDD.scala:53
No data
